# TECHNICAL REPORT: VRP with Increased Capacity (80 kg)

## 1. Introduction and Scenario Change

This experiment addresses the **Vehicle Routing Problem with Capacity (CVRP)** under a new operational constraint. Unlike the previous scenario, the fleet capacity has been increased to **80 kg**.

**Experiment Objective:**
To analyze how the capacity increase affects **customer feasibility** and **route efficiency**. Specifically, we seek to integrate the customer with the highest demand (C5), who was previously excluded, and minimize the total distance traveled while serving 100% of the customer portfolio.

## 2. Data Analysis and Constraints

### 2.1. Demand Profile
The calculated demands for the 5 customers are:

| Customer | Demand (kg) | Status (Cap. 60) | Status (Cap. 80) |
| :---: | :---: | :---: | :---: |
| C1 | 18.6 | Valid | **Valid** |
| C2 | 22.8 | Valid | **Valid** |
| C3 | 48.0 | Valid | **Valid** |
| C4 | 60.0 | Valid | **Valid** |
| C5 | **77.7** | *Invalid* | **Valid** |

### 2.2. Impact of New Capacity
The shift to 80 kg has a critical impact:
1.  **Total Inclusion:** Customer 5 (77.7 kg) now meets the constraint ($77.7 \le 80$). The Genetic Algorithm (GA) must now optimize a route for **5 customers** instead of 4.
2.  **Load Saturation:** Customer 5 will occupy 97% of a truck's capacity, practically forcing an exclusive trip for them (as only 2.3 kg remain free and no other customer has such small demand).
3.  **Combinatorics:** The extra capacity allows for new groupings. For example, C4 (60 kg) now leaves 20 kg free, allowing it to be grouped with C1 (18.6 kg), something impossible with the 60 kg capacity.

## 3. Methodology (Genetic Algorithm)

The GA configuration is maintained to ensure comparability:
* **Representation:** Permutation of indices of all valid customers.
* **Evaluation (Fitness):** Total Euclidean distance.
* **Route Decoding:** "Next Fit" Heuristic (fill the vehicle until the next customer does not fit, return to depot, and start new route).
* **Parameters:** 200 Generations, Population of 50 individuals.

--- 
### Code Implementation
*Below are the code cells for the experiment execution.*

In [ ]:
import random
import math
import matplotlib.pyplot as plt

# Problem data
depot = [20, 120]
clientes = [
    [35, 115], [50, 140], [70, 100], [40, 80], [25, 60]
]

pesos = [1.2, 3.8, 7.5, 0.9, 15.4, 12.1, 4.3, 19.7, 8.6, 2.5]

pedidos = [
    [(3, 2), (1, 3)],
    [(2, 6)],
    [(7, 4), (5, 2)],
    [(3, 8)],
    [(6, 5), (9, 2)]
]

In [ ]:
# 3. Demand Calculation and Capacity Definition
demandas = []
for pedido in pedidos:
    total = 0
    for item, cant in pedido:
        total += pesos[item - 1] * cant
    demandas.append(total)

# HERE IS THE KEY CHANGE: Capacity increased to 80
capacidad = 80

print('Calculated demands:', [round(d,1) for d in demandas])
print(f'Vehicle capacity: {capacidad} kg')

In [ ]:
# 4. Customer filtering
clientes_validos = []
demandas_validas = []
clientes_invalidos = []
demandas_invalidas = []

for i in range(len(clientes)):
    if demandas[i] <= capacidad:
        clientes_validos.append(clientes[i])
        demandas_validas.append(demandas[i])
    else:
        clientes_invalidos.append(clientes[i])
        demandas_invalidas.append(demandas[i])

print(f'Valid customers: {len(clientes_validos)} (All included)')
print(f'Invalid customers: {len(clientes_invalidos)}')

In [ ]:
# 5. GA Functions (Distance, Fitness, Crossover, Mutation)
def distancia(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def dividir_en_rutas(ruta):
    rutas = []
    carga = 0
    actual = []
    for c in ruta:
        if carga + demandas_validas[c] <= capacidad:
            actual.append(c)
            carga += demandas_validas[c]
        else:
            rutas.append(actual)
            actual = [c]
            carga = demandas_validas[c]
    if actual:
        rutas.append(actual)
    return rutas

def distancia_total(ruta):
    rutas = dividir_en_rutas(ruta)
    total = 0
    for r in rutas:
        pos = depot
        for c in r:
            total += distancia(pos, clientes_validos[c])
            pos = clientes_validos[c]
        total += distancia(pos, depot)
    return total

def fitness(ruta):
    return distancia_total(ruta)

def crear_poblacion(n, n_clientes):
    poblacion = []
    base = list(range(n_clientes))
    for _ in range(n):
        r = base[:]
        random.shuffle(r)
        poblacion.append(r)
    return poblacion

def seleccion(poblacion, fitnesses):
    i1, i2 = random.sample(range(len(poblacion)), 2)
    return poblacion[i1][:] if fitnesses[i1] < fitnesses[i2] else poblacion[i2][:]

def cruce(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    hijo = [-1] * len(p1)
    hijo[a:b] = p1[a:b]
    pos = b
    for x in p2:
        if x not in hijo:
            if pos >= len(p1): pos = 0
            hijo[pos] = x
            pos += 1
    return hijo

def mutacion(ruta, prob=0.1):
    if random.random() < prob:
        i, j = random.sample(range(len(ruta)), 2)
        ruta[i], ruta[j] = ruta[j], ruta[i]

def algoritmo_genetico(n_generaciones=200, tam_pob=50):
    poblacion = crear_poblacion(tam_pob, len(clientes_validos))
    fitnesses = [fitness(r) for r in poblacion]

    for _ in range(n_generaciones):
        nueva = []
        for _ in range(tam_pob):
            p1 = seleccion(poblacion, fitnesses)
            p2 = seleccion(poblacion, fitnesses)
            hijo = cruce(p1, p2)
            mutacion(hijo, 0.1)
            nueva.append(hijo)
        poblacion = nueva
        fitnesses = [fitness(r) for r in poblacion]

    mejor = min(range(tam_pob), key=lambda i: fitnesses[i])
    return poblacion[mejor], fitnesses[mejor]

## 4. Execution and Results Analysis

When executing the algorithm with the new capacity, very efficient grouping behavior is observed. The results are shown below:

In [ ]:
mejor_ruta, mejor_valor = algoritmo_genetico()

print('Best sequence (indices):', mejor_ruta)
print('Total distance:', round(mejor_valor, 2))

print("\n--- Detail of Generated Routes ---")
rutas = dividir_en_rutas(mejor_ruta)
for i, r in enumerate(rutas):
    carga = sum(demandas_validas[c] for c in r)
    uso_capacidad = (carga / capacidad) * 100
    print(f'Trip {i+1}: Customers {r} -> Load: {round(carga,1)} kg ({round(uso_capacidad, 1)}% occupancy)')

In [ ]:
def plot_rutas(rutas):
    plt.figure(figsize=(8, 8))
    # Draw customers
    for i, (x, y) in enumerate(clientes_validos):
        plt.scatter(x, y, c='blue', zorder=5)
        plt.text(x+1, y+1, f'C{i+1} ({round(demandas_validas[i],1)}kg)', fontsize=9)

    # Draw depot
    plt.scatter(depot[0], depot[1], c='red', s=100, marker='s', label='Depot', zorder=5)

    colores = ['green', 'orange', 'purple', 'cyan']
    
    for i, r in enumerate(rutas):
        puntos = [depot] + [clientes_validos[c] for c in r] + [depot]
        xs = [p[0] for p in puntos]
        ys = [p[1] for p in puntos]
        carga = sum(demandas_validas[c] for c in r)
        plt.plot(xs, ys, '-o', color=colores[i % len(colores)], label=f'Trip {i+1} ({round(carga,1)}kg)')

    plt.title(f"Optimal Routes (Capacity {capacidad}kg)")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

plot_rutas(rutas)

## 5. Final Experiment Conclusions (Capacity 80 kg)

The increase in vehicle capacity from 60 kg to **80 kg** has radically transformed the logistical solution. The main conclusions are:

1.  **Total Viability (C5 Included):**
    * The most critical factor is the inclusion of **Customer 5** (77.7 kg). By increasing capacity, this customer, which previously represented a lost sale or a logistical exception, now falls within standard operational parameters.
    
2.  **Load Optimization (High Filling Rate):**
    * A much more efficient **fleet saturation** is observed.
    * The combination of **Customer 4** (60 kg) with **Customer 1** (18.6 kg) totals **78.6 kg**. This represents an occupancy of **98.2%**, an ideal scenario in logistics that was mathematically impossible with the 60 kg constraint.
    * Customer 5 occupies **97.1%** of a vehicle on their own, justifying a dedicated trip.

3.  **Route Consolidation:**
    * Unlike the previous scenario where vehicles traveled with partial loads (e.g., 48 kg of 60 = 80%), here more merchandise is moved (all customers) with almost total use of available space on each trip.
    * This demonstrates that a **33% increase in capacity** (from 60 to 80) not only allows serving larger customers but drastically improves the combinatorial efficiency of the rest of the fleet.